<a href="https://colab.research.google.com/github/sumanth-github/mini_transformer/blob/main/llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install chromadb
!pip install tiktoken


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.7 MB/s eta 

In [5]:
!ngrok config add-authtoken 2x2czTq2vshQKZ0WK5yTPHJM7Ml_2KSCazfnhEDPtWrnuKHom

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.3 MB/s eta 0:00:00


In [4]:
!pip install pyngrok

In [23]:
from pyngrok import ngrok
import streamlit as st

# Write your app.py content
with open("app.py", "w") as f:
    f.write('''
import streamlit as st

st.title("Tiny LLM Chatbot")
user_input = st.text_input("You: ")

if user_input:
    # Dummy response logic, replace with real model
    st.write("Bot:", "This is a placeholder response.")
''')

# Start the Streamlit app
!streamlit run app.py &>/content/logs.txt &

# Expose it with ngrok
public_url = ngrok.connect(8501)
print("Your app is live at:", public_url)


Your app is live at: NgrokTunnel: "https://1e7602bd534a.ngrok-free.app" -> "http://localhost:8501"


In [7]:
%%writefile model.py
import torch
import torch.nn as nn

# Positional Encoding Layer
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=512):
        super().__init__()
        self.d_model = d_model
        self.max_len = max_len

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.pe = pe.unsqueeze(0)  # [1, max_len, d_model] but NOT registered buffer

    def forward(self, x):
        return self.pe[:, :x.size(1), :].to(x.device)


# One Transformer Encoder Layer
class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.attn = nn.MultiheadAttention(d_model, num_heads, dropout=dropout, batch_first=True)
        self.ff = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model)
        )
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        attn_out, _ = self.attn(x, x, x)
        x = self.norm1(x + self.dropout(attn_out))
        ff_out = self.ff(x)
        return self.norm2(x + self.dropout(ff_out))

# Full Transformer Encoder (stack of layers)
class TransformerEncoder(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads, d_ff, num_layers, max_seq_len):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(d_model, max_seq_len)
        self.layers = nn.ModuleList([
            TransformerEncoderLayer(d_model, num_heads, d_ff) for _ in range(num_layers)
        ])

    def forward(self, x):
        x = self.embedding(x)
        x = x + self.pos_encoding(x)
        for layer in self.layers:
            x = layer(x)
        return x

# TinyQA Model: Encoder + Decoder (CLS-style output)
class TinyQAModel(nn.Module):
    def __init__(self, encoder, d_model, vocab_size):
        super().__init__()
        self.encoder = encoder
        self.decoder = nn.Linear(d_model, vocab_size)  # Project [CLS] token output to vocab

    def forward(self, x):
        encoded = self.encoder(x)           # shape: (batch, seq_len, d_model)
        return self.decoder(encoded)      # output: (batch, vocab_size)


Writing model.py


In [8]:
%%writefile utils.py
import torch
from chromadb import Client
import chromadb.utils.embedding_functions as embedding_functions

from sentence_transformers import SentenceTransformer

# Initialize embedder
embedder = SentenceTransformer('all-MiniLM-L6-v2')
chroma_client = Client()
chroma_collection = chroma_client.get_or_create_collection("rag_memory")

def embed_chunks(chunks):
    embeddings = embedder.encode(chunks, show_progress_bar=True)
    ids = [f"id_{i}" for i in range(len(chunks))]
    chroma_collection.add(documents=chunks, embeddings=embeddings, ids=ids)
    return len(chunks)

def build_rag_prompt(user_query, top_k=3):
    from sentence_transformers import SentenceTransformer
    import chromadb
    chroma_client = chromadb.Client()
    collection = chroma_client.get_or_create_collection("rag_memory")

    embedder = SentenceTransformer("all-MiniLM-L6-v2")
    query_embedding = embedder.encode([user_query])[0]

    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )

    context_chunks = results["documents"][0]
    context = "\n\n".join(context_chunks)

    return f"Context:\n{context}\n\nQuestion: {user_query}\nAnswer:"



class SimpleTokenizer:
    def __init__(self):
        self.word2idx = {"<PAD>": 0, "<UNK>": 1}
        self.idx2word = {0: "<PAD>", 1: "<UNK>"}
        self.vocab_size = 2

    def fit(self, texts):
        for text in texts:
            for word in text.strip().split():
                if word not in self.word2idx:
                    idx = len(self.word2idx)
                    self.word2idx[word] = idx
                    self.idx2word[idx] = word
        self.vocab_size = len(self.word2idx)

    def encode(self, text, max_len=None):
        tokens = [self.word2idx.get(word, 1) for word in text.strip().split()]
        if max_len:
            tokens = tokens[:max_len] + [0] * max(0, max_len - len(tokens))
        return tokens

    def decode(self, tokens):
        return " ".join([self.idx2word.get(token, "<UNK>") for token in tokens])

def prepare_data(data, tokenizer, max_len):
    X = [tokenizer.encode(q, max_len=max_len) for q, a in data]
    y = [tokenizer.encode(a, max_len=max_len) for q, a in data]
    return torch.tensor(X, dtype=torch.long), torch.tensor(y, dtype=torch.long)


class BPETokenizer:
    def __init__(self, model_file="chatbot_bpe.model"):
        import sentencepiece as spm
        self.sp = spm.SentencePieceProcessor()
        self.sp.load(model_file)

    def encode(self, text, max_len=None):
        tokens = self.sp.encode(text, out_type=int)
        if max_len:
            tokens = tokens[:max_len] + [0] * max(0, max_len - len(tokens))
        return tokens

    def decode(self, token_ids):
        return self.sp.decode(token_ids)

    @property
    def vocab_size(self):
        return self.sp.get_piece_size()



Writing utils.py


In [9]:
import pickle

In [17]:
from google.colab import files
uploaded = files.upload()


Saving S08_question_answer_pairs.txt to S08_question_answer_pairs.txt


In [18]:
# Step 1: Open the file
with open("S08_question_answer_pairs.txt", "r") as f:
    lines = f.readlines()

# Step 2: Parse TSV and write questions + answers to one-line-per-sentence format
with open("bpe_train.txt", "w") as f_out:
    for line in lines[1:]:  # skip the header
        parts = line.strip().split('\t')
        if len(parts) >= 3:  # enough columns
            question = parts[1].strip()
            answer = parts[2].strip()
            if question and answer:
                f_out.write(question + "\n")
                f_out.write(answer + "\n")


In [20]:
!pip install sentencepiece
import sentencepiece as spm

spm.SentencePieceTrainer.train(
    input="bpe_train.txt",
    model_prefix="chatbot_bpe",
    vocab_size=800,
    model_type="bpe",
    character_coverage=1.0
)


In [21]:
%%writefile train.py
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from model import TransformerEncoder, TinyQAModel
from utils import BPETokenizer, prepare_data

# ==== Sample dataset ====
data = [
    ("Hi", "Hello!"),
    ("Hey there", "Hi!"),
    ("How are you?", "I'm doing well, thanks!"),
    ("What's your name?", "I'm your chatbot."),
    ("What can you do?", "I answer questions."),
    ("Who made you?", "A student learning AI."),
    ("Tell me a joke", "Why did the computer get cold? It left its Windows open!"),
    ("Nice joke", "Glad you liked it!"),
    ("Do you like pizza?", "I love data... but pizza sounds good too."),
    ("What is AI?", "Artificial Intelligence."),
    ("What is ML?", "Machine Learning."),
    ("Tell me something cool", "Transformers power modern AI like ChatGPT!"),
    ("Can you help me?", "Of course. What do you need?"),
    ("Are you alive?", "Not yet 😄"),
    ("Bye", "Goodbye!"),
    ("Thanks", "You're welcome!"),
    ("What’s 2 + 2?", "It’s 4."),
    ("What's the capital of India?", "New Delhi."),
    ("Can you sing?", "I can rhyme in binary."),
    ("What is Python?", "A programming language."),
]


# ==== Hyperparameters ====
max_len = 100
d_model = 32
num_heads = 2
d_ff = 64
num_layers = 2
batch_size = 2
num_epochs = 10

# ==== Tokenizer & Data ====
tokenizer = BPETokenizer("chatbot_bpe.model")

#
X, y = prepare_data(data, tokenizer, max_len)
dataset = TensorDataset(X, y)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# ==== Model ====
vocab_size = tokenizer.vocab_size
encoder = TransformerEncoder(vocab_size, d_model, num_heads, d_ff, num_layers, max_seq_len=max_len)
model = TinyQAModel(encoder, d_model, vocab_size)

# ==== Training ====
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_history = []
for epoch in range(num_epochs):
    total_loss = 0
    for batch_x, batch_y in loader:
        logits = model(batch_x)  # shape: [batch, seq_len, vocab_size]
        logits = logits.view(-1, vocab_size)  # [batch * seq_len, vocab_size]
        targets = batch_y.view(-1)            # [batch * seq_len]
        loss = criterion(logits, targets)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    avg_loss = total_loss / len(loader)
    loss_history.append(avg_loss)
    print(f"Epoch {epoch+1}: Loss = {avg_loss:.4f}")

with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)
# ==== Save model (optional) ====
torch.save(model.state_dict(), "tinyqa_model.pth")
print("✅ Training complete. Model saved.")


Overwriting train.py


In [22]:
!python train.py


2025-07-23 19:12:40.342724: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753297960.406628    6590 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753297960.421747    6590 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-23 19:12:40.480929: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
modules.json: 100% 349/349 [00:00<00:00, 1.77MB/s]
config_sentence_transformers.json: 100% 116/116 [00:00<00:00, 599kB/s]
REA

In [13]:
%%writefile inference.py
import torch
import pickle
from model import TransformerEncoder, TinyQAModel
from utils import BPETokenizer

# ==== Load tokenizer & hyperparams ====
with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)



max_len = 100
d_model = 32
num_heads = 2
d_ff = 64
num_layers = 2

# ==== Load model ====
vocab_size = tokenizer.vocab_size
encoder = TransformerEncoder(vocab_size, d_model, num_heads, d_ff, num_layers, max_seq_len=max_len)
model = TinyQAModel(encoder, d_model, vocab_size)
state_dict = torch.load("tinyqa_model.pth")
state_dict = {k: v for k, v in state_dict.items() if not k.startswith("encoder.pos_encoding.pe")}
model.load_state_dict(state_dict, strict=False)
model.eval()

# ==== Inference function ====
def answer_question(prompt: str):
    input_ids = tokenizer.encode(prompt, max_len)
    input_tensor = torch.tensor([input_ids], dtype=torch.long)

    with torch.no_grad():
        logits = model(input_tensor)             # (1, T, V)
        output_ids = logits[0].argmax(dim=-1)    # (T,)
        tokens = [t.item() for t in output_ids if t.item() != tokenizer.sp.pad_id()]
        return tokenizer.decode(tokens)



def generate_response(prompt: str, max_len=20):
    input_ids = tokenizer.encode(prompt, max_len)
    input_tensor = torch.tensor([input_ids], dtype=torch.long)

    response_tokens = []
    with torch.no_grad():
        for _ in range(max_len):
            logits = model(input_tensor)
            next_token_id = logits[0, -1].argmax().item()

            if next_token_id == tokenizer.sp.pad_id():
                break

            response_tokens.append(next_token_id)
            input_tensor = torch.cat(
                [input_tensor, torch.tensor([[next_token_id]])], dim=1
            )

    return tokenizer.decode(response_tokens)


def load_model(model_path="tinyqa_bpe_model.pth", tokenizer_path="chatbot_bpe.model", max_len=100, d_model=32, num_heads=2, d_ff=64, num_layers=2):
    tokenizer = BPETokenizer(tokenizer_path)
    vocab_size = tokenizer.vocab_size

    encoder = TransformerEncoder(vocab_size, d_model, num_heads, d_ff, num_layers, max_seq_len=max_len)
    model = TinyQAModel(encoder, d_model, vocab_size)

    state_dict = torch.load(model_path)
    state_dict = {k: v for k, v in state_dict.items() if not k.startswith("encoder.pos_encoding.pe")}
    model.load_state_dict(state_dict, strict=False)
    model.eval()

    return model, tokenizer


# ==== Test ====
if __name__ == "__main__":
    while True:
        q = input("\nAsk me anything (or type 'exit'): ")
        if q.lower() == 'exit':
            break
        print("🤖", answer_question(q))


Writing inference.py


In [24]:
%%writefile app.py
import streamlit as st
import pickle
import json
import datetime
from inference import answer_question,generate_response,load_model
from utils import BPETokenizer,build_rag_prompt




# ------------------ Session State ------------------
def init_session():
    if "history" not in st.session_state:
        st.session_state.history = []

# ------------------ Sidebar ------------------
def render_sidebar():
    st.sidebar.title("🧠 Tiny Transformer")
    st.sidebar.markdown("Crafted by: **Sumanth Kadarla**")
    st.sidebar.markdown("🎓 B.Tech CSE (AIML), Tier-3")
    st.sidebar.markdown("📍 India | 🚀 FAANG-bound")
    st.sidebar.markdown("---")
    return st.sidebar.radio("📂 Navigate", [
        "🏠 Home",
        "💬 QA Chat",
        "🗨️ Chat Mode",
        "📜 History",
        "📚 Dataset",
        "📊 Training Monitor",
        "⚙️ Model Settings",
        "🧠 Architecture",
        "📘 About"
    ])
    st.sidebar.subheader("📂 Load Saved Model")
    model_file = st.sidebar.file_uploader("Upload Model (.pth)", type="pth")
    tokenizer_file = st.sidebar.file_uploader("Upload Tokenizer (.model)", type="model")

    if model_file and tokenizer_file:
        with open("uploaded_model.pth", "wb") as f:
            f.write(model_file.read())
        with open("uploaded_tokenizer.model", "wb") as f:
            f.write(tokenizer_file.read())

        st.session_state.custom_model = "uploaded_model.pth"
        st.session_state.custom_tokenizer = "uploaded_tokenizer.model"
        st.sidebar.success("✅ Model & Tokenizer loaded!")


# ------------------ Home ------------------
def render_home():
    st.title("🏠 Welcome to TinyTransformer QA Playground")
    st.markdown("""
    A minimal transformer-based chatbot project to learn how GPT-like models work under the hood.

    - Built from scratch using PyTorch
    - Streamlit-powered dashboard
    - Simple token-based Q&A transformer
    """)
    st.markdown("---")
    st.subheader("🧭 Navigate using the sidebar to explore features")

# ------------------ Chat ------------------
def render_qa_chat():
    st.title("💬 Ask a Question")
    tab1, tab2 = st.tabs(["🧠 Ask", "🔍 Explanation (coming soon)"])

    with tab1:
        user_input = st.text_input("Type your question:")
        if st.button("🎯 Get Answer") and user_input:
            prompt = build_rag_prompt(user_input)
            answer = answer_question(prompt).strip()
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            st.session_state.history.append({
                "question": user_input,
                "answer": answer,
                "timestamp": timestamp
            })
            st.success(f"🤖 {answer}")

    with tab2:
        st.info("Model explanation and token-wise breakdown will go here in a future update!")


def render_chatbot():
  st.title("🗨️ Chatbot Mode")
  example = st.selectbox("🧪 Try an Example", [
    "", "What is AI?", "Who is Elon Musk?", "Tell me a joke", "What is Python?"
  ])

  if example:
      st.session_state.chat_input = example


  if "chat_history" not in st.session_state:
      st.session_state.chat_history = []

  user_input = st.text_input("You:", value=st.session_state.get("chat_input", ""), key="chat_input")
  if st.button("Send"):
    if not user_input.strip():
      st.warning("Please enter a message to chat.")
      return
    prompt = build_rag_prompt(user_input)
    reply = answer_question(prompt)
    st.session_state.chat_history.append({"user": user_input, "bot": reply})
    st.markdown(f"<div style='background-color:#f0f2f6; padding:10px; border-radius:10px'><b>You:</b> {user_input}</div>", unsafe_allow_html=True)
    st.markdown(f"<div style='background-color:#e0ffe0; padding:10px; border-radius:10px'><b>Bot:</b> {reply}</div>", unsafe_allow_html=True)







# ------------------ History ------------------
def render_history():
    st.title("📜 Conversation History")
    if st.session_state.history:
        for item in st.session_state.history[::-1]:
            with st.expander(f"🕒 {item['timestamp']}"):
                st.markdown(f"**Q:** {item['question']}")
                st.markdown(f"**A:** {item['answer']}")
    else:
        st.info("No conversation history yet.")

    if st.button("💾 Save Chat"):
        with open("chat_history.json", "w") as f:
            json.dump(st.session_state.history, f)
        st.success("Saved to chat_history.json")

    uploaded_file = st.file_uploader("📂 Load Previous History", type="json")
    if uploaded_file:
        st.session_state.history.extend(json.load(uploaded_file))
        st.success("History loaded.")

# ------------------ Dataset ------------------
def render_dataset():
    import pandas as pd

    st.title("📚 Dataset Viewer")
    tab1, tab2 = st.tabs(["🔍 Preview", "📥 Upload"])

    with tab1:
        st.markdown("### 📄 Dataset Preview")

        if "uploaded_df" in st.session_state:
            st.success("Showing uploaded dataset:")
            st.dataframe(st.session_state.uploaded_df)
        else:
            st.info("No dataset uploaded yet. Showing static preview instead.")
            st.table([
                {"Question": "What is AI?", "Answer": "Artificial Intelligence"},
                {"Question": "Who is Elon Musk?", "Answer": "Entrepreneur"},
                {"Question": "What is Python?", "Answer": "Programming"},
            ])

    with tab2:
        import chromadb
        from sentence_transformers import SentenceTransformer

        st.header("📂 Upload & Embed Your Knowledge")

        if "doc_chunks" not in st.session_state:
            st.session_state.doc_chunks = []

        embedder = SentenceTransformer("all-MiniLM-L6-v2")
        chroma_client = chromadb.Client()
        if "rag_memory" not in [c.name for c in chroma_client.list_collections()]:
            collection = chroma_client.create_collection("rag_memory")
        else:
            collection = chroma_client.get_collection("rag_memory")

        file = st.file_uploader("Upload a .txt file for knowledge base", type=["txt"])
        if file:
            text = file.read().decode("utf-8")
            chunks = text.split("\n\n")  # basic paragraph split
            st.session_state.doc_chunks = chunks
            st.success(f"✅ Loaded {len(chunks)} text chunks.")

        if st.button("🔍 Embed into ChromaDB"):
            chunks = st.session_state.get("doc_chunks", [])
            if chunks:
                embeddings = embedder.encode(chunks, show_progress_bar=True)
                ids = [f"id_{i}" for i in range(len(chunks))]
                collection.add(documents=chunks, embeddings=embeddings, ids=ids)
                st.success(f"✅ {len(chunks)} chunks embedded into ChromaDB!")
            else:
                st.warning("Please upload a document first.")


# ------------------ Training Monitor ------------------
def render_training():
    import pandas as pd
    import torch
    import torch.nn as nn
    import torch.optim as optim
    import pickle
    from torch.utils.data import DataLoader, TensorDataset
    from utils import SimpleTokenizer, prepare_data
    from model import TransformerEncoder, TinyQAModel
    from sklearn.model_selection import train_test_split

    st.title("📊 Train Model Inside Streamlit")

    if "uploaded_df" not in st.session_state:
        st.warning("Please upload a dataset first in the 📚 Dataset tab.")
        return

    df = st.session_state.uploaded_df
    df.columns = [col.lower() for col in df.columns]

    # Detect columns
    if "question" in df.columns and "answer" in df.columns:
        q_col, a_col = "question", "answer"
    elif "text" in df.columns and "response" in df.columns:
        q_col, a_col = "text", "response"
    else:
        st.error("Dataset must contain either ['question', 'answer'] or ['text', 'response']")
        return

    st.subheader("⚙️ Training Configuration")
    epochs = st.slider("Epochs", 1, 20, 5)
    batch_size = st.selectbox("Batch Size", [2, 4, 8, 16], index=1)
    lr = st.number_input("Learning Rate", value=1e-3, format="%.5f")
    max_len = st.slider("Max Token Length", 10, 100, 50)
    st.subheader("⚙️ Transformer Hyperparameters")

# 🧩 These are OUTSIDE the if-condition
    d_model = st.slider("Model Size (d_model)", 16, 128, 32, step=16)
    num_heads = st.slider("Number of Attention Heads", 1, 8, 2)
    num_layers = st.slider("Number of Encoder Layers", 1, 6, 2)
    d_ff = st.slider("Feedforward Hidden Dim (d_ff)", 32, 256, 64, step=32)
    max_len = st.slider("Max Sequence Length", 10, 100, 50)

    # Optional model summary live preview
    st.code(f"""
    TinyQAModel(
      encoder = TransformerEncoder(
        layers = {num_layers},
        heads = {num_heads},
        d_model = {d_model},
        d_ff = {d_ff},
        max_len = {max_len}
      ),
      decoder = nn.Linear({d_model} → vocab)
    )
    """, language="python")

    if st.button("🚀 Train Model"):
        df = df[df[q_col].notna() & df[a_col].notna()]
        df = df.astype({q_col: str, a_col: str})
        uploaded_data = list(zip(df[q_col], df[a_col]))

        chat_data = [
            ("Hi", "Hello!"),
            ("Hey there", "Hi!"),
            ("How are you?", "I'm doing well, thanks!"),
            ("What's your name?", "I'm your chatbot."),
            ("What can you do?", "I answer questions."),
            ("Who made you?", "A student learning AI."),
            ("Tell me a joke", "Why did the computer get cold? It left its Windows open!"),
            ("Nice joke", "Glad you liked it!"),
            ("Do you like pizza?", "I love data... but pizza sounds good too."),
            ("What is AI?", "Artificial Intelligence."),
            ("What is ML?", "Machine Learning."),
            ("Tell me something cool", "Transformers power modern AI like ChatGPT!"),
            ("Can you help me?", "Of course. What do you need?"),
            ("Are you alive?", "Not yet 😄"),
            ("Bye", "Goodbye!"),
            ("Thanks", "You're welcome!"),
            ("What’s 2 + 2?", "It’s 4."),
            ("What's the capital of India?", "New Delhi."),
            ("Can you sing?", "I can rhyme in binary."),
            ("What is Python?", "A programming language.")
        ]


        # Combine base and uploaded dataset
        combined_data = chat_data + uploaded_data

        # ⛔ Clean
        combined_data = [(q, a) for q, a in combined_data if isinstance(q, str) and isinstance(a, str)]

        # ✅ Split
        train_data, val_data = train_test_split(combined_data, test_size=0.2, random_state=42)

        # Fit tokenizer on **all** data
        tokenizer = BPETokenizer("chatbot_bpe.model")
        # Prepare tensors
        X_train, y_train = prepare_data(train_data, tokenizer, max_len)
        X_val, y_val = prepare_data(val_data, tokenizer, max_len)

        train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=batch_size)

        vocab_size = tokenizer.vocab_size

        encoder = TransformerEncoder(vocab_size, d_model, num_heads, d_ff, num_layers, max_seq_len=max_len)
        model = TinyQAModel(encoder, d_model, vocab_size)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)
        loss_history = []

        progress_bar = st.progress(0)
        status_text = st.empty()
        train_loss_history = []
        val_loss_history = []
        val_acc_history = []

        for epoch in range(epochs):
          model.train()
          total_loss = 0
          for batch_x, batch_y in train_loader:
              logits = model(batch_x)
              logits = logits.view(-1, vocab_size)
              targets = batch_y.view(-1)

              loss = criterion(logits, targets)
              loss.backward()
              optimizer.step()
              optimizer.zero_grad()
              total_loss += loss.item()

          avg_train_loss = total_loss / len(train_loader)
          train_loss_history.append(avg_train_loss)

          # 🔍 VALIDATION
          model.eval()
          val_loss = 0
          correct = 0
          total = 0
          with torch.no_grad():
              for val_x, val_y in val_loader:
                  logits = model(val_x)
                  logits = logits.view(-1, vocab_size)
                  targets = val_y.view(-1)
                  loss = criterion(logits, targets)
                  val_loss += loss.item()

                  preds = torch.argmax(logits, dim=1)
                  correct += (preds == targets).sum().item()
                  total += targets.size(0)

          avg_val_loss = val_loss / len(val_loader)
          val_acc = correct / total

          val_loss_history.append(avg_val_loss)
          val_acc_history.append(val_acc)

          progress_bar.progress((epoch + 1) / epochs)
          status_text.text(
              f"Epoch {epoch + 1}/{epochs} | "
              f"Train Loss: {avg_train_loss:.4f} | "
              f"Val Loss: {avg_val_loss:.4f} | "
              f"Val Acc: {val_acc:.2%}"
          )
        torch.save(model.state_dict(), "tinyqa_model.pth")
        with open("tokenizer.pkl", "wb") as f:
            pickle.dump(tokenizer, f)
        misclassified = []

        with torch.no_grad():
            for val_x, val_y in val_loader:
                logits = model(val_x)
                preds = torch.argmax(logits, dim=1)
                for x, y_true, y_pred in zip(val_x, val_y, preds):
                    if y_true != y_pred:
                        question = tokenizer.decode([t for t in x.tolist() if t != 0])
                        answer = tokenizer.decode([y_true.item()])
                        predicted = tokenizer.decode([y_pred.item()])
                        misclassified.append((question, answer, predicted))

        if misclassified:
            st.subheader("❌ Misclassified Examples")
            for q, true_a, pred_a in misclassified[:5]:  # limit to 5 for speed
                st.markdown(f"**Q:** {q}")
                st.markdown(f"**True A:** {true_a} | **Pred A:** {pred_a}")
                st.markdown("---")

        st.success("✅ Model trained and saved!")
        st.subheader("📉 Training Progress")

        chart_data = {
            "Train Loss": train_loss_history,
            "Val Loss": val_loss_history,
            "Val Accuracy": val_acc_history
        }
        st.line_chart(chart_data)
        model_name = st.text_input("Model name to save (no extension)", "tinyqa_bpe")
        if st.button("💾 Save Model"):
            torch.save(model.state_dict(), f"{model_name}.pth")
            with open(f"{model_name}_tokenizer.model", "wb") as f:
                f.write(open("chatbot_bpe.model", "rb").read())
            st.success(f"Model and tokenizer saved as {model_name}.pth and {model_name}_tokenizer.model")





# ------------------ Settings ------------------
def render_settings():
    st.title("⚙️ Model Settings")
    model_path = st.session_state.get("custom_model", "tinyqa_bpe_model.pth")
    tokenizer_path = st.session_state.get("custom_tokenizer", "chatbot_bpe.model")


    st.info(f"""
    **Current Model**: `{model_path}`
    **Tokenizer**: `{tokenizer_path}`
    """)

    if st.button("🔁 Reset to Default"):
        st.session_state.custom_model = "tinyqa_bpe_model.pth"
        st.session_state.custom_tokenizer = "chatbot_bpe.model"
        st.success("Reset to default model.")
# ------------------ Architecture ------------------
def render_architecture():
    st.title("🧠 Model Architecture")
    st.code("""
TinyQAModel(
  encoder = TransformerEncoder(...),
  decoder = nn.Linear(d_model → vocab)
)
    """, language="python")
    st.markdown("Heads: 2 | Layers: 2 | d_model: 32 | max_len: 5")

# ------------------ About ------------------
def render_about():
    st.title("📘 About This App")
    st.markdown("""
This app is a **mini Transformer QA bot** built from scratch using PyTorch.

**Built by:** Sumanth
**Degree:** B.Tech CSE (AIML), Tier-3
**Current Focus:** ML Engineering | Strategic AI Roles
    """)
    st.info("Built for learning and showcasing LLM mechanics. Updates coming soon!")
    st.markdown("---")
    st.markdown("💡 [GitHub Repo](#) | 🧠 Powered by PyTorch + Streamlit + SentencePiece")

# ------------------ Main ------------------
def main():
    st.set_page_config(page_title="TinyQA Bot", layout="wide", initial_sidebar_state="collapsed")
    init_session()
    section = render_sidebar()

    if section == "🏠 Home":
        render_home()
    elif section == "💬 QA Chat":
        render_qa_chat()
    elif section == "🗨️ Chat Mode":
      render_chatbot()
    elif section == "📜 History":
        render_history()
    elif section == "📚 Dataset":
        render_dataset()
    elif section == "📊 Training Monitor":
        render_training()
    elif section == "⚙️ Model Settings":
        render_settings()
    elif section == "🧠 Architecture":
        render_architecture()
    elif section == "📘 About":
        render_about()

if __name__ == "__main__":
    main()


Overwriting app.py


In [15]:
!pip install sentencepiece